In [1]:
import torch
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import torchtext
import unicodedata

In [2]:
##Change working directory to talk-berty-to-me root
import os
os.chdir("D:/University/Projects/AML/talk-berty-to-me")

In [ ]:
# data_gutenberg = pd.read_csv('data/books_and_genres.csv')
# dataset_train = data_gutenberg.sample(frac=0.6, random_state=0)
# dataset_val = data_gutenberg.drop(dataset_train.index).sample(frac=0.5, random_state=0)
# dataset_test = data_gutenberg.drop(dataset_train.index).drop(
#     dataset_val.index)
# dataset_train.to_parquet('data/datasets/train.parquet', index=False)
# dataset_test.to_parquet('data/datasets/test.parquet', index=False)
# dataset_val.to_parquet('data/datasets/val.parquet', index=False)
# data_gutenberg.sample(20).to_parquet('data/datasets/dev.parquet', index=False)

In [ ]:
#Add code to switch between datasets here

In [ ]:
# dev_data = pd.read_parquet('data/datasets/dev.parquet')

In [3]:
def contains_accents(input_str):
    decomposed = unicodedata.normalize('NFD', input_str)
    return not all([unicodedata.category(char) != 'Mn' for char in decomposed])

In [4]:
raw_data = pd.read_csv('data/books_and_genres_eng.csv')


In [5]:
raw_data.loc[:,'is_accent'] = raw_data.loc[:,'text'].apply(contains_accents)


In [6]:
raw_data = raw_data.loc[raw_data.loc[:,'is_accent'] == False, :]

In [7]:
raw_data = raw_data.drop_duplicates(subset=['title'])

In [8]:
raw_data.head()

,Unnamed: 0,title,text,genres,language_code,is_accent
0,31,the power and the glory,"Produced by Juliet Sutherland, Sjaani and PG D...","{'literary-fiction', 'christian', 'history', '...",eng,False
1,32,paradise,Produced by Judith Smith and Natalie Salter\n\...,"{'literary-fiction', 'mythology', 'historical-...",eng,False
4,147,persuasion,Produced by Sharon Partridge and Martin Ward. ...,"{'romance', 'literary-fiction', 'classics', 'h...",eng,False
7,157,alcestis,"Produced by Ted Garvin, Charles M. Bidwell and...","{'romance', 'literary-fiction', 'history', 'cl...",eng,False
8,292,dead souls,Produced by John Bickers\n\n\n\n\n\nDEAD SOULS...,"{'literary-fiction', 'roman', 'classics', 'nov...",eng,False


In [9]:
dev_data = raw_data.sample(20)
dev_data.drop(columns=['Unnamed: 0'], inplace=True)

###Creating a validation set
val_data = raw_data.sample(4)
val_data.drop(columns=['Unnamed: 0'], inplace=True)

In [10]:
dev_data.head()

,title,text,genres,language_code,is_accent
97,no name,Produced by James Rusk and David Widger\n\n\n\...,"{'romance', 'literary-fiction', 'crime', 'myst...",eng,False
366,the scarlet pimpernel,Produced by Conway Yee and David Widger\n\n\n\...,"{'romance', 'adventure', 'literary-fiction', '...",en-US,False
303,creatures of the night,"Produced by David Edwards, Marcia Brooks and t...","{'horror', 'animals', 'comics', 'fantasy', 'pa...",eng,False
390,the frogs,"Produced by Ted Garvin, Marvin A. Hodges, Char...","{'classics', 'mythology', 'literature', 'ficti...",en-US,False
139,after dark,Produced by James Rusk\n\n\n\n\n\nAFTER DARK\n...,"{'contemporary', 'literary-fiction', '21st-cen...",eng,False


In [11]:
val_data.head()

,title,text,genres,language_code,is_accent
390,the frogs,"Produced by Ted Garvin, Marvin A. Hodges, Char...","{'classics', 'mythology', 'literature', 'ficti...",en-US,False
148,eastern standard tribe,Eastern Standard Tribe\n\nCory Doctorow\n\nCop...,"{'literary-fiction', 'science-fiction', 'fanta...",en-US,False
92,the tale of the flopsy bunnies,Produced by Michael Ciesielski and the Online ...,"{'picture-books', 'animals', 'classics', 'fant...",eng,False
220,the untouchable,"Produced by Greg Weeks, David Wilson and the O...","{'literary-fiction', 'history', 'novella', 'hi...",eng,False


In [12]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
def clean_text(text):
    '''
    Function to clean text of books. Removes email addresses, new lines, html tags, and extra spaces.

    Input: Text (String)
    Output: Cleaned Text (String)
    '''
    cleaned_text = text.lower()
    cleaned_text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' ', text)
    cleaned_text = re.sub(r'^.*?(?=\n\n\n)', ' ', cleaned_text, flags=re.DOTALL)
    cleaned_text = re.sub(r'<a\s+(?:[^>]*?\s+)?href="([^"]*)"[^>]*>.*?</a>', ' ', cleaned_text, flags=re.DOTALL)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'\d+', '', cleaned_text)
    cleaned_text = re.sub(r'[^\w\s.?!]', ' ', cleaned_text)
    cleaned_text = re.sub(r' +', ' ', cleaned_text)
    return cleaned_text

def remove_first_row(group):
    return group.iloc[1:]

def lowercase(text):
    return text.lower()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\setul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\setul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
dev_data['cleaned_text'] = dev_data['text'].apply(lambda x:clean_text(x))
dev_data.loc[:,'genres'] = dev_data.loc[:,'genres'].apply(lambda x:clean_text(x))
dev_data.loc[:,'sentences'] = dev_data.loc[:,'cleaned_text'].apply(lambda x: nltk.tokenize.sent_tokenize(str(x)))
dev_data = dev_data.explode('sentences')
dev_data.loc[:,'sentences'] = dev_data.loc[:,'sentences'].apply(lambda x:lowercase(x))
dev_data.reset_index(drop=True, inplace=True)
dev_data['label_sentences'] = dev_data.groupby('title')['sentences'].shift(-1)
dev_data = dev_data.dropna(subset=['label_sentences'])
dev_data = dev_data.loc[:,['title', 'genres','sentences', 'label_sentences']]

In [14]:
val_data['cleaned_text'] = val_data['text'].apply(lambda x:clean_text(x))
val_data.loc[:,'genres'] = val_data.loc[:,'genres'].apply(lambda x:clean_text(x))
val_data.loc[:,'sentences'] = val_data.loc[:,'cleaned_text'].apply(lambda x: nltk.tokenize.sent_tokenize(str(x)))
val_data = val_data.explode('sentences')
val_data.loc[:,'sentences'] = val_data.loc[:,'sentences'].apply(lambda x:lowercase(x))
val_data.reset_index(drop=True, inplace=True)
val_data['label_sentences'] = val_data.groupby('title')['sentences'].shift(-1)
val_data = val_data.dropna(subset=['label_sentences'])
val_data = val_data.loc[:,['title', 'genres','sentences', 'label_sentences']]

In [15]:
dev_data = dev_data.loc[:,['title', 'genres','sentences', 'label_sentences']]
dev_data.head()

,title,genres,sentences,label_sentences
0,no name,romance literary fiction crime mystery classi...,no name by wilkie collins editorial note ital...,the main purpose of this story is to appeal to...
1,no name,romance literary fiction crime mystery classi...,the main purpose of this story is to appeal to...,here is one more book that depicts the struggl...
2,no name,romance literary fiction crime mystery classi...,here is one more book that depicts the struggl...,it has been my aim to make the character of ma...
3,no name,romance literary fiction crime mystery classi...,it has been my aim to make the character of ma...,this design was no easy one to accomplish and ...
4,no name,romance literary fiction crime mystery classi...,this design was no easy one to accomplish and ...,round the central figure in the narrative othe...


In [16]:
val_data = val_data.loc[:,['title', 'genres','sentences', 'label_sentences']]
val_data.head()

,title,genres,sentences,label_sentences
0,the frogs,classics mythology literature fiction comedy ...,the harvard classics edited by charles w elio...,he is credited with over forty plays eleven of...
1,the frogs,classics mythology literature fiction comedy ...,he is credited with over forty plays eleven of...,his satire deal with political religious and l...
2,the frogs,classics mythology literature fiction comedy ...,his satire deal with political religious and l...,the attic comedy was produced at the festivals...
3,the frogs,classics mythology literature fiction comedy ...,the attic comedy was produced at the festivals...,aristophanes seems indeed to have been regarde...
4,the frogs,classics mythology literature fiction comedy ...,aristophanes seems indeed to have been regarde...,he died shortly after the production of his pl...


In [17]:
dev_data.loc[1905, 'sentences']

'now norah!'

In [ ]:
# #Building vocabulary original and functional
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator

# tokenizer = get_tokenizer('basic_english')
# vocab_iter = iter(dev_data.loc[:,'sentences'] + dev_data.loc[:,'title'] + dev_data.loc[:,'genres'])
# def yield_tokens(train_iter):
#     for text in train_iter:
#         if not isinstance(text, str):
#             if type(text) == list:
#                 for t in text:
#                     yield tokenizer(t)
#             continue
#         yield tokenizer(text)

In [28]:
#Building vocabulary fixed
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
vocab_iter = iter(dev_data.loc[:,'sentences'])
def yield_tokens(train_iter):
    for text in train_iter:
        if not isinstance(text, str):
            if type(text) == list:
                for t in text:
                    yield tokenizer(t)
            continue
        yield tokenizer(text)

In [29]:
vocab = build_vocab_from_iterator(
    yield_tokens(vocab_iter), specials=["<unk>", "<pad>", "<BOS>", "<EOS>"], min_freq=25)
vocab.set_default_index(vocab["<unk>"])

In [30]:
len(vocab)

6276

In [31]:
VECTOR_CACHE_DIR = '/Users/setul/mlpp23/.vector_cache'
glove = torchtext.vocab.GloVe('6B', cache=VECTOR_CACHE_DIR)
glove_vectors = glove.get_vecs_by_tokens(vocab.get_itos())

In [32]:
dev_data.iloc[2480:2486]

,title,genres,sentences,label_sentences
2480,no name,romance literary fiction crime mystery classi...,if the aspect of mr. vanstone s character whic...,his next words raised the veil and showed the ...
2481,no name,romance literary fiction crime mystery classi...,his next words raised the veil and showed the ...,chapter xiii.
2482,no name,romance literary fiction crime mystery classi...,chapter xiii.,the fortune which mr. vanstone possessed when ...
2483,no name,romance literary fiction crime mystery classi...,the fortune which mr. vanstone possessed when ...,mr. vanstone the elder was a manufacturer in t...
2484,no name,romance literary fiction crime mystery classi...,mr. vanstone the elder was a manufacturer in t...,he married early in life and the children of t...
2485,no name,romance literary fiction crime mystery classi...,he married early in life and the children of t...,first michael the eldest son still living and ...


In [33]:
import torch.nn as nn
import torch.nn.functional as F
IS_CUDA = torch.cuda.is_available()
if IS_CUDA:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [34]:
from torch.nn.utils.rnn import pad_sequence
def collate_batch(batch):
    titles, genres, sentences, label_sentences = zip(*batch)
    #Add a separator tag between title and genre
    context = [tokenizer(g) + ['<pad>'] + tokenizer(t) + ['<pad>'] +
               ['<BOS>'] + tokenizer(s) + ['<EOS>'] for t, g,
                s in zip(titles, genres, sentences)]
    label_sentence = [['<BOS>'] + tokenizer(s) + ['<EOS>'] for s in label_sentences]
    label_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in label_sentence],
                                    padding_value=vocab['<pad>'], batch_first=True)
    encoder_tensor = pad_sequence([torch.tensor(vocab.lookup_indices(t)) for t in context],
                                 padding_value=vocab['<pad>'], batch_first=True)
    return encoder_tensor, label_tensor

In [35]:
#From HW4
from torch.utils.data import Sampler
class BatchSequentialSampler(Sampler):
    r"""Samples batches, s.t. the ith elements of each batch are sequential.

    Args:
        data_source (Dataset): dataset to sample from
    """

    def __init__(self, data_source, batch_size):
        self.data_source = data_source
        self.batch_size = batch_size
        
    def __iter__(self):
        num_batches = len(self.data_source)//self.batch_size
        for i in range(num_batches):
            for j in range(self.batch_size):
                yield(j * num_batches + i)

    def __len__(self):
        return (len(self.data_source)//self.batch_size) * self.batch_size

In [36]:
batch_sampler = BatchSequentialSampler(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']], 8)

In [37]:
batch_dataloader = torch.utils.data.DataLoader(dev_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']].values,
                                                   batch_size=8, collate_fn=collate_batch, sampler=batch_sampler)

In [38]:
val_sampler = BatchSequentialSampler(val_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']], 8)

In [39]:
val_dataloader = torch.utils.data.DataLoader(val_data.loc[:,['title', 'genres', 'sentences', 'label_sentences']].values,
                                                   batch_size=8, collate_fn=collate_batch, sampler=val_sampler)

In [40]:
for idx, (context_tensor, label_tensor) in enumerate(batch_dataloader):
    print(context_tensor.shape)
    print(label_tensor.shape)
    break

torch.Size([8, 83])
torch.Size([8, 107])


In [41]:
class BiRNN_encoder(nn.Module):
    def __init__ (self, embedding_dim, hidden_dim,
                  vocab_size, num_layers=2, type_rnn = 'LSTM', bidirectional = True,
                  dropout = 0.3, pad_idx = 0):
        super(BiRNN_encoder, self).__init__()
        self.rnns = []
        # self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx).to(device)
        self.embedding = nn.Embedding.from_pretrained(glove_vectors, freeze=False).to(device)
        hidden_size = hidden_dim * 2 if bidirectional else hidden_dim
        input_size = embedding_dim
        for _ in range(num_layers):
            if type_rnn == 'LSTM':
                rnn = nn.LSTM(input_size, hidden_size, 1, dropout = dropout,
                               bidirectional = bidirectional, batch_first=True).to(device)
            elif type_rnn == 'GRU':
                rnn = nn.GRU(input_size, hidden_size, 1, dropout = dropout,
                              bidirectional = bidirectional, batch_first=True).to(device)
            self.rnns.append(rnn)
            input_size = hidden_size*2 if bidirectional else hidden_size
        self.rnns = nn.ModuleList(self.rnns).to(device)
        self.dropout = nn.Dropout(dropout).to(device)
        self.type_rnn = type_rnn
        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, input, hidden = None):
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
        rnn_input = embedded
        for idx, rnn in enumerate(self.rnns):
            output, hidden_output = rnn(rnn_input, hidden)
            hidden = hidden_output
            rnn_input = output
        if self.type_rnn == 'LSTM' and self.bidirectional:
            hidden_state = torch.cat((hidden[0][-2,:,:], hidden[0][-1,:,:]), dim = 1).to(device)
            cell = torch.cat((hidden[1][-2,:,:], hidden[1][-1,:,:]), dim = 1).to(device)
            hidden = (hidden_state, cell)
        elif self.type_rnn == 'GRU' and self.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1).to(device)
        return hidden

In [42]:
encoder = BiRNN_encoder(300, 600, len(vocab), num_layers=4, type_rnn = 'LSTM',
                         bidirectional = True, dropout = 0.3, pad_idx = vocab['<pad>']).to(device)

c:\Users\setul\miniconda3\envs\talk-berty\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [43]:
for i, batch in enumerate(batch_dataloader):
    context_tensor, label_tensor = batch
    context_tensor, label_tensor = context_tensor.to(device), label_tensor.to(device)
    hidden,cell = encoder(context_tensor)
    print(hidden.shape)
    print(cell.shape)
    break

torch.Size([8, 2400])
torch.Size([8, 2400])


In [44]:
class BiRNN_decoder(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, vocab_dim, dropout = 0.3):
        super(BiRNN_decoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.vocab_dim = vocab_dim
        self.dropout = nn.Dropout(dropout).to(device)
        # self.embedding = nn.Embedding(vocab_dim, embedding_dim).to(device)
        self.embedding = nn.Embedding.from_pretrained(glove_vectors, freeze=False).to(device)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout = dropout).to(device)
        self.fc_out = nn.Linear(hidden_dim, vocab_dim).to(device)


    def forward(self, input, hidden, context):
        embedded = self.dropout(self.embedding(input))
        outputs, (hidden,context) = self.rnn(embedded, (hidden, context))
        predictions = self.fc_out(outputs)
        predictions = predictions.squeeze(0)
        return predictions, hidden, context
    


In [45]:
decoder = BiRNN_decoder(300, 2400, 1, len(vocab), dropout = 0.3).to(device)

In [46]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, trg, hidden, teacher_ratio = 0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = len(vocab)
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        hidden, context = self.encoder(src, hidden)
        # hidden = hidden.detach()
        # context = context.detach()
        dec_input = trg[:, 0]
        dec_input = dec_input.unsqueeze(0)
        hidden = hidden.unsqueeze(0)
        context = context.unsqueeze(0)
        for t in range(1, trg_len):
            output, hidden, context = self.decoder(dec_input, hidden, context)
            outputs[t] = output
            top1 = output.argmax(1)
            dec_input = trg[:,t] if np.random.random() < teacher_ratio else top1
            dec_input = dec_input.unsqueeze(0)
        return outputs

In [47]:
USE_CUDA = True

In [48]:
def evaluate(model, data_loader):
    '''
    Evaluate the model on the given data.
    '''

    model.eval()
    it = iter(data_loader)
    total_count = 0. # Number of target words seen
    total_loss = 0. # Loss over all target words
    with torch.no_grad():
        # No gradients need to be maintained during evaluation
        # There are no hidden tensors for the first batch, and so will default to zeros.
        hidden = None 
        for i, batch in enumerate(it):
            text, target = batch
            text, target = text.to(device), target.to(device)
            output = model(text, target, hidden).to(device)
            mask = (target != PAD_IDX)
            ntotal = mask.sum()
            loss = loss_fn(output.view(-1, output.size(-1)), target.view(-1))
            total_count += ntotal
            total_loss += loss.item()/ntotal
                
    # loss = total_loss / total_count
    model.train()
    return loss

In [ ]:
# ##ChatGPT's improvement
# def evaluate(model, data_loader):
#     model.eval()
#     total_loss = 0.0
#     total_count = 0  # Total number of non-<pad> tokens

#     with torch.no_grad():
#         for text, target in data_loader:
#             if USE_CUDA:
#                 text, target = text.cuda(), target.cuda()

#             output = model(text, target)
#             mask = (target != PAD_IDX)  # Create a mask for non-pad tokens
#             ntotal = mask.sum().item()  # Sum the mask values to get the total number of non-<pad> tokens

#             # Apply mask to filter out loss contributions from <pad> tokens and compute the loss
#             loss = loss_fn(output.view(-1, output.size(-1)), target.view(-1))
#             loss = loss.masked_select(mask.view(-1)).mean()  # Only consider non-<pad> tokens and compute mean loss
            
#             total_loss += loss.item() * ntotal  # Accumulate the total loss
#             total_count += ntotal  # Accumulate the total count of non-<pad> tokens

#     average_loss = total_loss / total_count  # Compute the average loss over all non-<pad> tokens
#     model.train()
#     return average_loss

In [50]:
LOG_FILE = "language-model.log"
GRAD_CLIP = 1.
NUM_EPOCHS = 10
PAD_IDX = vocab['<pad>']

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if h is None:
        return None
    elif isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    
    
seq_model = Seq2Seq(encoder, decoder).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_IDX, reduction = 'sum') ## Used instead of NLLLoss.
learning_rate = 0.001
optimizer = torch.optim.Adam(seq_model.parameters(), lr=learning_rate)
val_losses = []
best_model = None
for epoch in range(NUM_EPOCHS):
    seq_model.train()
    it = iter(batch_dataloader)
    hidden = None
    for i, batch in enumerate(it):
        data, target = batch
        data, target = data.to(device), target.to(device)
        hidden = repackage_hidden(hidden)
        output = seq_model(data, target, hidden).to(device)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[:,1:].reshape(-1)
        optimizer.zero_grad()
        mask = (target != PAD_IDX)
        #ntotal = mask.sum()
        loss = loss_fn(output, target)
        # loss = loss[mask]
        # loss = loss.mean()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(seq_model.parameters(), GRAD_CLIP)
        optimizer.step()


        if i % 1000 == 0:
            print("epoch", epoch, "iter", i, "loss", loss.item())

        if i % 10000 == 0:
            val_loss = evaluate(seq_model, val_dataloader)
            with open(LOG_FILE, "a") as fout:
                print("epoch: {}, iteration: {}".format(epoch, i))
                fout.write("epoch: {}, iteration: {}, perplexity: {}\n".format(epoch, i, val_loss))

            # Save the model if the validation loss is the minimum so far
            if len(val_losses) == 0 or val_loss < min(val_losses):
                print("best model, val loss: ", val_loss)
                # #best_model = copy.deepcopy(model)
                # best_model = type(seq_model)(vocab_size, EMBEDDING_SIZE, EMBEDDING_SIZE, 2, dropout=0.5)
                # if USE_CUDA:
                #     best_model = best_model.cuda()
                # best_model.load_state_dict(model.state_dict())

                # with open(PATH + "lm-best.th", "wb") as fout:
                #     torch.save(best_model.state_dict(), fout)
            else:
                learning_rate /= 4.
                optimizer = torch.optim.Adam(seq_model.parameters(), lr=learning_rate)
            val_losses.append(val_loss)


epoch 0 iter 0 loss 2167.66357421875
epoch: 0, iteration: 0
best model, val loss:  tensor(516.3895, device='cuda:0')
epoch 0 iter 1000 loss 1071.380615234375
epoch 0 iter 2000 loss 611.8499145507812
epoch 0 iter 3000 loss 857.4074096679688
epoch 0 iter 4000 loss 933.5927734375
epoch 0 iter 5000 loss 895.7481689453125
epoch 0 iter 6000 loss 1260.715576171875
epoch 0 iter 7000 loss 785.7858276367188
epoch 0 iter 8000 loss 1048.9405517578125


In [ ]:
# ##From other sources
# epoch_loss = 0.0
# num_epochs = 10
# best_loss = 999999
# best_epoch = -1
# sentence1 = "Hello I am starting"
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss(ignore_index = vocab['<pad>'])
# ts1 = []
# for epoch in range(num_epochs):
#   print("Epoch - {} / {}".format(epoch+1, num_epochs))
#   model.train(True)
#   for batch_idx, batch in enumerate(batch_dataloader):
#     input , target = batch
#     input, target = input.to(device), target.to(device)
#     output = model(input, target).to(device)
#     output = output[1:].reshape(-1, output.shape[2])
#     target = target[:,1:].reshape(-1)
    
#     optimizer.zero_grad()
#     loss = criterion(output, target)
#     loss.backward()

#     # Clip gradient >1 to prevent exploding gradients
#     torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

#     # Update the weights values using the gradients we calculated using bp 
#     optimizer.step()
#     #step += 1
#     epoch_loss += loss.item()
#     #writer.add_scalar("Training loss", loss, global_step=step)

#   if epoch_loss < best_loss:
#     best_loss = epoch_loss
#     best_epoch = epoch
#     if ((epoch - best_epoch) >= 3):
#       print("no improvement in 3 epochs, break")
#       break
#   print("Epoch_Loss - {}".format(loss.item()))
#   print()
  
# print(epoch_loss / len(batch_dataloader))

In [ ]:
torch.save(seq_model.state_dict(), 'seq_model_3_layers.pt')

In [ ]:
model = torch.load('seq_model_functional.pt')

In [ ]:
def generate_text(model, input_genre, input_title, input_text, max_length=10):
    model.eval()

    # Tokenize and prepare input text
    input_tokens = (tokenizer(input_genre) + ['<pad>'] + tokenizer(input_title) + ['<pad>'] +
    ['<BOS>'] + tokenizer(input_text) + ['<EOS>'])
    input_indices = vocab.lookup_indices(input_tokens)
    input_tensor = torch.tensor([input_indices], dtype=torch.long, device=device)

    with torch.no_grad():
        hidden, context = model.encoder(input_tensor)
    decoder_input = torch.tensor([[vocab['<BOS>']]], device=device)  
    output_indices = []
    context = context.unsqueeze(0)
    hidden = hidden.unsqueeze(0)

    for _ in range(max_length):
        with torch.no_grad():
            output, hidden, context = model.decoder(decoder_input, hidden, context)
            output_probabilities = output.squeeze().softmax(dim = -1).to(device)
            top1 = torch.multinomial(output_probabilities, 1)[0]
            if top1.item() == vocab['<EOS>']:
                break
            output_indices.append(top1.item())
            decoder_input = torch.tensor([[top1.item()]], device=device)  
    output_tokens = [vocab.lookup_token(index) for index in output_indices]
    return ' '.join(output_tokens)


In [ ]:
def generate_text(model, input_genre, input_title, input_text, max_length=10):
    model.eval()

    # Tokenize and prepare input text
    input_tokens = (tokenizer(input_genre) + ['<pad>'] + tokenizer(input_title) + ['<pad>'] +
    ['<BOS>'] + tokenizer(input_text) + ['<EOS>'])
    input_indices = vocab.lookup_indices(input_tokens)
    input_tensor = torch.tensor([input_indices], dtype=torch.long, device=device)

    with torch.no_grad():
        hidden, context = model.encoder(input_tensor)
    decoder_input = torch.tensor([[vocab['<BOS>']]], device=device)  
    output_indices = []
    context = context.unsqueeze(0)
    hidden = hidden.unsqueeze(0)

    for _ in range(max_length):
        with torch.no_grad():
            output, hidden, context = model.decoder(decoder_input, hidden, context)
            output_probabilities = output.squeeze().softmax(dim = -1).to(device)
            #top1 = torch.multinomial(output_probabilities, 1)[0]
            topk_values, topk_indices = torch.topk(output_probabilities, k=10)
            # print(topk_values)
            # print(topk_indices)
            top1 = torch.multinomial(topk_values, 1)[0]
            if top1.item() == vocab['<BOS>'] or top1.item() ==vocab['<unk>'] or top1.item() ==vocab['<pad>']:
                continue
            if top1.item() == vocab['<EOS>'] and len(output_indices) >3:
                break
            elif top1.item() == vocab['<EOS>']:
                continue
            output_indices.append(top1.item())
            decoder_input = torch.tensor([[top1.item()]], device=device)  
    output_tokens = [vocab.lookup_token(index) for index in output_indices]
    return ' '.join(output_tokens)

In [ ]:
input_title = "the game"
input_genre = "fiction"
input_sentence = "he saw a line."
generated_text = generate_text(seq_model, input_genre, input_title, input_sentence, max_length=50)
print(generated_text)

In [ ]:
for param in seq_model.parameters():
    print("param", param.grad)